In [79]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from sklearn.metrics import average_precision_score
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
#from kaggle_submission import output_submission_csv
from classifier import Classifier
#from classifier1 import Classifier1
from preprocessing import preprocess, VOC_CLASSES
from preprocessing import DummySampler
from collections import defaultdict,Counter
from scipy.spatial import distance
from sklearn.metrics import f1_score

import pickle
import ast

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Transforms applied to the training data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std= [0.229, 0.224, 0.225])

train_transform = transforms.Compose([
           
            transforms.RandomHorizontalFlip(),
            #transforms.RandomRotation(45),
            transforms.RandomResizedCrop(224,scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333), interpolation=2),
          
            transforms.Resize(227),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            normalize,
           
            
        ])


In [109]:
import pandas as pd
csvfile=pd.read_csv('./final_test_data.csv')
csvfile=csvfile.drop_duplicates()

In [110]:

test_dir = './Data/test_photos_new'
train_imgs = os.listdir(test_dir)

In [111]:
test_transform = transforms.Compose([
            transforms.Resize(227),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            normalize,
        ])

In [112]:
class_list=[]

In [113]:
vector_quant=defaultdict()
def train_classifier(train_loader, classifier, criterion, optimizer):
    classifier.train()
    loss_ = 0.0
    losses = []
    #print("hello")
    for i, (images, labels,business_id) in enumerate(train_loader):
        #print(labels.size())
        classifier.eval()
        images, labels = images.to(device), labels.to(device)
        logits = classifier(images)
        logits=logits.detach()
        logits=np.array(logits)
        labels=np.array(labels)
        #print(logits.shape)
        logit_average=np.average(logits,axis=0)
        label_average=np.average(labels,axis=0)
        class_list.append(repr(list(label_average)))
        if repr(list(label_average)) not in vector_quant:
            vector_quant[repr(list(label_average))]=logit_average
        else:    
            vector_quant[repr(list(label_average))]=vector_quant[repr(list(label_average))]+logit_average
        #print(label_average)
        #print(logit_average.shape)                    
        #loss = criterion(logits, labels)
        #print(loss)
        #loss.backward()
        #optimizer.step()
        #losses.append(loss)
        #print(losses)
        #print(logit_average)
    print("Done")    
    #return #torch.stack(losses).mean().item()

In [39]:
label_average=np.array([1,2,3,3,4])
repr(label_average)

'array([1, 2, 3, 3, 4])'

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [114]:
ds_train = preprocess('./Data/test_photos_new',train_transform,csvfile)

0.0000 percent is completed
7.9650 percent is completed
15.9299 percent is completed
23.8949 percent is completed
31.8598 percent is completed
39.8248 percent is completed
47.7897 percent is completed
55.7547 percent is completed
63.7196 percent is completed
71.6846 percent is completed
79.6495 percent is completed
87.6145 percent is completed
95.5795 percent is completed
['471926.jpg', '53875.jpg', '221363.jpg', '383074.jpg', '41403.jpg', '102511.jpg', '330023.jpg', '85843.jpg', '59533.jpg', '229812.jpg', '226479.jpg', '5054.jpg', '440073.jpg', '457070.jpg', '138904.jpg', '451923.jpg', '381551.jpg', '1232.jpg', '385027.jpg', '150276.jpg', '64637.jpg', '388690.jpg', '174591.jpg', '116833.jpg', '466480.jpg', '238269.jpg', '380103.jpg', '165637.jpg', '214734.jpg', '325344.jpg', '90930.jpg', '223209.jpg', '424179.jpg', '262361.jpg', '345214.jpg', '12327.jpg', '86383.jpg', '30333.jpg', '105070.jpg', '29728.jpg', '100265.jpg', '302529.jpg', '254753.jpg', '30000.jpg', '53320.jpg', '99191.jpg

In [128]:
sampler = DummySampler(ds_test.business_id)

In [115]:
train_loader = torch.utils.data.DataLoader(dataset=ds_train,
                                                
                                               batch_size=70,
                                               sampler=sampler,
                                                
                                               num_workers=0)


In [30]:
classifier = Classifier().to(device)
# You can can use this function to reload a network you have already saved previously
classifier.load_state_dict(torch.load('voc_classifier.pth',map_location=lambda storage, loc: storage))


In [116]:
import numpy as np
#arr=np.array([1,1,1,3,1,1,1,1,1,4,1,2,1,1,1,3,3,2,1,2,1])
#weight=torch.from_numpy(arr)
#weight=weight.to(device)

criterion = nn.MultiLabelSoftMarginLoss()
#optimizer = torch.optim.SGD(classifier.parameters(), lr=0.001, momentum=0.92)
optimizer =torch.optim.Adam(classifier.parameters(), lr=0.0001, betas=(0.9, 0.999))

In [117]:
# Training the Classifier
NUM_EPOCHS = 10
TEST_FREQUENCY = 20
num_iteration=100

for epoch in range(1, NUM_EPOCHS+1):
    print("Starting epoch number " + str(epoch))
    #train_loss=[]
    for i in range(num_iteration):
   
        
        train_classifier(train_loader, classifier, criterion, optimizer)
        
    #print("Loss for Training on Epoch " +str(epoch) + " is "+ str(train_loss/num_iteration))
    '''if(epoch%TEST_FREQUENCY==0):
            mAP_val, val_loss, _ = test_classifier(val_loader, classifier, criterion)
            print('Evaluating classifier')
            print("Mean Precision Score for Testing on Epoch " +str(epoch) + " is "+ str(mAP_val))'''

Starting epoch number 1
3706
	calling Sampler:__iter__
Done
2548
	calling Sampler:__iter__
Done
3668
	calling Sampler:__iter__
Done
465
	calling Sampler:__iter__
Done
3377
	calling Sampler:__iter__
Done
985
	calling Sampler:__iter__
Done
439
	calling Sampler:__iter__
Done
742
	calling Sampler:__iter__
Done
3897
	calling Sampler:__iter__
Done
1631
	calling Sampler:__iter__
Done
3380
	calling Sampler:__iter__
Done
3888
	calling Sampler:__iter__
Done
3847
	calling Sampler:__iter__
Done
507
	calling Sampler:__iter__
Done
2535
	calling Sampler:__iter__
Done
3380
	calling Sampler:__iter__
Done
3858
	calling Sampler:__iter__
Done
645
	calling Sampler:__iter__
Done
2779
	calling Sampler:__iter__
Done
3216
	calling Sampler:__iter__
Done
3606
	calling Sampler:__iter__
Done
242
	calling Sampler:__iter__
Done
795
	calling Sampler:__iter__
Done
32
	calling Sampler:__iter__
Done
917
	calling Sampler:__iter__
Done
415
	calling Sampler:__iter__
Done
491
	calling Sampler:__iter__
Done
3216
	calling Sam

In [118]:
val_quant=defaultdict()
valu=Counter(class_list)
for k,v in vector_quant.items():
    val_quant[k]=vector_quant[k]/valu[k]

In [119]:
val_quant


defaultdict(None,
            {'[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]': array([-0.884815  , -0.90661776, -0.56283283,  0.5538913 , -2.048828  ,
                     0.22294435,  0.5341182 , -2.169511  ,  1.4982114 ], dtype=float32),
             '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]': array([-1.6187226 ,  0.05919117,  0.49847984,  0.77118677, -0.7731682 ,
                     0.9668051 ,  1.2645    , -1.1120818 ,  0.41587037], dtype=float32),
             '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]': array([-0.9747882 , -0.5852115 ,  0.027648  ,  0.49609584, -1.7440114 ,
                     0.625615  ,  1.1277446 , -1.7613906 ,  1.10112   ], dtype=float32),
             '[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]': array([-1.1650157 , -0.4718136 , -0.03623262,  0.6228997 , -1.374595  ,
                     0.6637463 ,  0.85634273, -1.7888823 ,  0.8764118 ], dtype=float32),
             '[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]': array([-0.7895322 , -0.92881

In [138]:
def test_classifier(test_loader, classifier, print_ind_classes=True):
    classifier.eval()
    losses = []
    classes=list(val_quant.keys())
    #f1Score=[]
    
    with torch.no_grad():
        #y_true = np.zeros((0,21))
        #y_score = np.zeros((0,21))
        for i, (images, labels,_) in enumerate(test_loader):
            images, labels = images.to(device), labels.to(device)
            logits = classifier(images)
            #y_true = np.concatenate((y_true, labels.cpu().numpy()), axis=0)
            #y_score = np.concatenate((y_score, logits.cpu().numpy()), axis=0)
            #loss = criterion(logits, labels)
            #losses.append(loss)
            logits=np.array(logits)
            labels=np.array(labels)
            #print(labels)
            logit_average=np.average(logits,axis=0)
            label_average=np.average(labels,axis=0)
            #print(label_average)
            d_value=[]
            for key,value in val_quant.items():
                dist=distance.euclidean(logit_average,value)
                d_value.append(dist)
            index=np.argmin(d_value)
            label_pred=eval(classes[index])
            #print(label_pred)
            #print(label_average)
            
            #f1_score(label_average,label_pred, average='macro')   
            
    #f1=np.mean(f1Score)
        
        
    return label_average, label_pred

In [121]:
test_csvfile=pd.read_csv('./final_val_data.csv')
test_csvfile=test_csvfile.drop_duplicates()

In [122]:
test_dir = './Data/val_photos_new'
test_imgs = os.listdir(test_dir)

In [130]:
ds_test = preprocess(test_dir, test_transform,test_csvfile)

test_loader = torch.utils.data.DataLoader(dataset=ds_test,
                                               batch_size=80, 
                                               sampler=sampler,
                                               num_workers=0)

0.0000 percent is completed
20.1694 percent is completed
40.3388 percent is completed
60.5083 percent is completed
80.6777 percent is completed
['407147.jpg', '324104.jpg', '177066.jpg', '11307.jpg', '124286.jpg', '293144.jpg', '449795.jpg', '181616.jpg', '233821.jpg', '410605.jpg', '263481.jpg', '91483.jpg', '198680.jpg', '302214.jpg', '375322.jpg', '80197.jpg', '278261.jpg', '431127.jpg', '305488.jpg', '153549.jpg', '247204.jpg', '352935.jpg', '217662.jpg', '205330.jpg', '192443.jpg', '229310.jpg', '129864.jpg', '108215.jpg', '88035.jpg', '376933.jpg', '255600.jpg', '461786.jpg', '180430.jpg', '415850.jpg', '216433.jpg', '259776.jpg', '273779.jpg', '305134.jpg', '265724.jpg', '403498.jpg', '227977.jpg', '423319.jpg', '201096.jpg', '96881.jpg', '126583.jpg', '355843.jpg', '467743.jpg', '28227.jpg', '108497.jpg', '161718.jpg', '321346.jpg', '409048.jpg', '120602.jpg', '382415.jpg', '424684.jpg', '181366.jpg', '104742.jpg', '120334.jpg', '111555.jpg', '173662.jpg', '320097.jpg', '423980

In [139]:
la_=[]
lp_=[]
for _ in range(200):
    
    la,lp= test_classifier(test_loader, classifier)
    la_.append(la)
    lp_.append(lp)
    

1327
	calling Sampler:__iter__
2730
	calling Sampler:__iter__
2748
	calling Sampler:__iter__
2515
	calling Sampler:__iter__
2060
	calling Sampler:__iter__
3789
	calling Sampler:__iter__
327
	calling Sampler:__iter__
609
	calling Sampler:__iter__
1910
	calling Sampler:__iter__
2592
	calling Sampler:__iter__
609
	calling Sampler:__iter__
2730
	calling Sampler:__iter__
803
	calling Sampler:__iter__
803
	calling Sampler:__iter__
2405
	calling Sampler:__iter__
3321
	calling Sampler:__iter__
2060
	calling Sampler:__iter__
1139
	calling Sampler:__iter__
2933
	calling Sampler:__iter__
147
	calling Sampler:__iter__
3506
	calling Sampler:__iter__
803
	calling Sampler:__iter__
1046
	calling Sampler:__iter__
1139
	calling Sampler:__iter__
1910
	calling Sampler:__iter__
968
	calling Sampler:__iter__
2592
	calling Sampler:__iter__
1490
	calling Sampler:__iter__
1654
	calling Sampler:__iter__
1654
	calling Sampler:__iter__
3426
	calling Sampler:__iter__
3999
	calling Sampler:__iter__
794
	calling Sam

In [140]:
actual=np.array(la_)
pred=np.array(lp_)


In [146]:
f1=f1_score(actual,pred,average='micro')

In [142]:
actual


array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 1., ..., 1., 0., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 0., 0.]], dtype=float32)

In [147]:
f1




0.777838727372463

In [145]:
f1_score(actual,pred,average=None)

array([0.46601942, 0.89915966, 0.86956522, 0.57416268, 0.58695652,
       0.85501859, 0.89864865, 0.71328671, 0.74545455])